## 1: Create Client

In [44]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv())

client: OpenAI = OpenAI()

## 2: Define Functions

In [45]:
import requests
import json
#user_input = input("Enter your crypto name")

def get_crypto(token_name:str):
    
    url = f"https://api.coincap.io/v2/assets/{token_name.lower()}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        data2 = json.loads(data['data'].get('supply'))
        return {"token_name" : f"{data2}"}
       
    else:
        print(f"Error: {response.status_code}")
        return response.text
get_crypto("bitcoin")

{'token_name': '19573218.0'}

In [46]:
import json

def show_json(message,obj):
    display(json.loads(message, obj.model_dump_json()))

In [47]:
from openai.types.beta.assistant import Assistant

assistant: Assistant = client.beta.assistants.create(
    model="gpt-3.5-turbo-1106",
    tools=[{
        "type":"function",
        "function": {
            "name" : "get_crypto",
            "description": "get the priceUsd in token_name",
            "parameters":{
                "type":"object",
                "properties": {
                    "token_name": {"type":"string", "description": "The token price in usd e.g. 29.98"},
                    #"priceUsd": {"type":"string"}     
                },
                "required":["token_name"]
            }
        }
        
    }]
)

In [48]:
from openai.types.beta.thread import Thread
from openai.types.beta.threads.thread_message import ThreadMessage
from openai.types.beta.threads.run import Run

thread: Thread = client.beta.threads.create()
print(thread)

message: ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Whats the usd price of bitcoin?"
)

dict(message)

run:Run = client.beta.threads.runs.create(
    assistant_id=assistant.id,
    thread_id=thread.id,
)
dict(run)



Thread(id='thread_rDADm9hkrg0OcuYWgrrLiEXt', created_at=1702879036, metadata={}, object='thread')


{'id': 'run_uN0Fd1ZzYaM33BoOap8k1rDA',
 'assistant_id': 'asst_pWCftn1j5pUL9wygDjzVXgE5',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1702879037,
 'expires_at': 1702879637,
 'failed_at': None,
 'file_ids': [],
 'instructions': None,
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_rDADm9hkrg0OcuYWgrrLiEXt',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='get_crypto', parameters={'type': 'object', 'properties': {'token_name': {'type': 'string', 'description': 'The token price in usd e.g. 29.98'}}, 'required': ['token_name']}, description='get the priceUsd in token_name'), type='function')]}

In [49]:
import time

available_functions = {"get_crypto": get_crypto}

while True:
    runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    runSteps = client.beta.threads.runs.steps.list(thread_id=thread.id, run_id=run.id)
    print(runStatus.status, "......")
    
    if runStatus == "requires_action":
        print(runStatus.status,'.....')
        print("status :","requires_action!")
        show_json("submit_tool_outputs", runStatus.required_action)
        
        if runStatus.required_action.submit_tool_outputs and runStatus.required_action.submit_tool_outputs.tool_calls:
            print("toolCalls presents")
            toolCalls = runStatus.required_action.submit_tool_outputs.tool_calls
            
            tool_outputs = []
            for toolCall in toolCalls:
                function_name = toolCall.function.name
                function_args = json.loads(toolCall.function.arguments)
                
                if function_name in available_functions:
                    function_to_call = available_functions[function_name]
                    print(function_to_call,function_to_call.__name__=="get_crypto","================================================================")
                    
                    if function_to_call.__name__ == "get_crypto":
                        response = function_to_call(
                            token_name=function_args.get("token_name")
                            )
                        
                        tool_outputs.append({
                            "tool_call_id":toolCall.id,
                            "output":response})
                        
            print(tool_outputs,">>>>>>>>")
            # Submit tool outputs and update the run
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tool_outputs)
            
    elif runStatus.status == "completed":
        # List the messages to get the response
        print("completed...........logic")
        messages: list[ThreadMessage] = client.beta.threads.messages.list(thread_id=thread.id)
        for message in messages.data:
            role_label = "User" if message.role == "user" else "Assistant"
            message_content = message.content[0].text.value
            print(f"{role_label}: {message_content}\n")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
      print("Run failed.")
      break

    elif run.status in ["in_progress", "queued"]:
      print(f"Run is {run.status}. Waiting...")
      time.sleep(5)  # Wait for 5 seconds before checking again

    else:
      print(f"Unexpected status: {run.status}")
      break
            
                        
                    

in_progress ......
Run is queued. Waiting...
requires_action ......
Run is queued. Waiting...
requires_action ......
Run is queued. Waiting...
requires_action ......
Run is queued. Waiting...
requires_action ......
Run is queued. Waiting...
requires_action ......
Run is queued. Waiting...
requires_action ......
Run is queued. Waiting...


KeyboardInterrupt: 